In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
# Import different modules for using with the notebook
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import HTML
from IPython.display import display
from IPython.display import Image

from sklearn.linear_model import LogisticRegression as logis
from sklearn.metrics import confusion_matrix

from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model, datasets

#from utils import plot_confusion_matrix
from matplotlib.colors import ListedColormap

from scipy.stats import norm
import math
import itertools

In [20]:
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [20]:
def sigmoid(x):
        return 1.0/(1 + np.exp(-x))

def cost(X, y, W):
        d,N = np.shape(X)
        ans = 0
        Wt = W.reshape((1,d))
        for n in range(N):
            sig = sigmoid(np.dot(Wt, X[:,n]))
            ans = ans + y[n] * np.log(sig) + (1-y[n]) * np.log(1-sig)  
        return -ans 
    
def gradient(X, y, W):
    
        #X = np.matrix([[-1,2,7],[-3,0,5]])
        #y = np.array([0,0,1])
    d,N = np.shape(X)
    Wt = W.reshape((1,d))
    
    grad = np.zeros((d,1))

    for n in range(N):
        x = X[:,n]
        xt = np.reshape(x, (d, 1))
        sig = sigmoid(np.dot(Wt, xt))
        sig1 = y[n]-sig
        grad = grad + (float(sig1))*xt 

            # grad = grad + np.dot((y[n]-sig),X[:,n])
    return -grad 

def hessian(X, y, W):
    
        #X = np.matrix([[-1,2,7],[-3,0,5]])
        #XT = X.T
        #y = np.matrix([[0,0,1]])
    d,N = np.shape(X)
    Wt = W.reshape((1,d))

    hes = np.zeros((d,d))


    for n in range(N):
        x = X[:,n].reshape((d,1))
        xt = np.reshape(x, (1, d))
        sig = sigmoid(np.dot(Wt,x))
        hes = hes + float(sig*(1-sig))*np.dot(x,xt)
             
    return hes 

def updateW(X, y, W):
        hessianInv = np.linalg.inv(hessian(X,y,W)) 
        grad = gradient(X,y,W)
        dot = np.dot(hessianInv, grad)
        W_new = W - dot
        #Wt_new = W_new.T
        #Wt = Wt - np.dot(hessianInv, grad)
        return W_new    

X = np.array([[0.1055, 0.7], [0.3, 0.1473]])
print(X)
y = np.array([[1], [0]])
print(y)
w = np.array([[1], [1]])
print(w)
print(cost(X,y,w))
print(gradient(X,y,w))
print(hessian(X,y,w))
print(updateW(X,y,w))

[[0.1055 0.7   ]
 [0.3    0.1473]]
[[1]
 [0]]
[[1]
 [1]]
[1.71478597]
[[ 0.4478012 ]
 [-0.01688742]]
[[0.10557115 0.02924903]
 [0.02924903 0.02615628]]
[[-5.4049112 ]
 [ 8.80787179]]


In [64]:
class LogisticReg:
    
    def ___init___(self):
#         self.lambd = 2
        self.X = None
        self.y = None
        self.N = None
        self.k = None
        self.d = None
        self.W_fin = None
        

    # sigmoid function
    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

#test sigmoid
#X = np.array([-1,2,-3,4])
#print(sigmoid(X))

#E(w) = -logp(D|w)
# def cost(X,y,wt):
#     d,N = np.shape(X) 
#     ans = 0
#     for i in range(N):
#         sig = sigmoid(np.dot(wt,X[:,i]))
#         ans = ans + (-y[0,i] * np.log(sig) - (1-y[0,i]) * np.log(1-sig))        
#     return ans

    def cost(self, W):
        lam = 2
#         d,N = np.shape(X)
        ans = 0
        Wt = W.reshape((1,self.d))
        for n in range(self.N):
            sig = self.sigmoid(np.dot(Wt, self.X[:,n]))
            ans = ans + self.y[n] * np.log(sig) + (1-self.y[n]) * np.log(1-sig)  
        return -ans + (1/(2*lam))*Wt*W

# #calc gradient
    def gradient(self, W):
        lam = 2
        #X = np.matrix([[-1,2,7],[-3,0,5]])
        #y = np.array([0,0,1])
        Wt = W.reshape((1,self.d))
#         d,N = np.shape(X)
        grad = np.zeros((self.d,1))

        for n in range(self.N):
            x = self.X[:,n]
            xt = np.reshape(x, (self.d, 1))
            sig = self.sigmoid(np.dot(Wt, xt))
            sig1 = self.y[n]-sig
            grad = grad + (float(sig1))*xt 

            # grad = grad + np.dot((y[n]-sig),X[:,n])
        return -grad + (1/lam)*W

    # #calculate Hessian
    def hessian(self, W):
        lam = 2
        #X = np.matrix([[-1,2,7],[-3,0,5]])
        #XT = X.T
        #y = np.matrix([[0,0,1]])
        Wt = W.reshape((1,self.d))
#         d,N = np.shape(X)
        hes = np.zeros((self.d,self.d))


        for n in range(self.N):
            x = self.X[:,n].reshape((self.d,1))
            xt = np.reshape(x, (1, self.d))
            sig = self.sigmoid(np.dot(Wt,x))
            hes = hes + float(sig*(1-sig))*np.dot(x,xt)
             
        return hes + ((1/lam)*np.diag(np.ones(self.d)))

    #update w using newtons method => wk+1 = wk - (grad/hes)
    def updateW(self, W):
        hessianInv = np.linalg.inv(self.hessian(W)) 
        grad = self.gradient(W)
        dot = np.dot(hessianInv, grad)
        W_new = W - dot
        #Wt_new = W_new.T
        #Wt = Wt - np.dot(hessianInv, grad)
        return W_new


    def fit(self, X, y):
        
        self.d,self.N = np.shape(X)
        print(self.d)
        ones = np.array([np.ones(self.N)])
        self.X = np.concatenate((ones,X),axis = 0)
        self.d,self.N = np.shape(self.X)
        print(self.d)
        
        self.y = y
        
        w_old = np.ones(self.d)
        print(w_old)
        w_new = np.ones(self.d)
        print(w_new)
        
#         w_old = np.matrix([[0.5],[1],[1]])
        perc = 1
        while (perc > 0.01):
            w_new = self.updateW(w_old)
            perc = np.linalg.norm(w_old - w_new)/np.linalg.norm(w_old)
            w_old = w_new
            self.W_fin = w_new
        
    
    def predict(self, X_p):
        self.d, self.N = np.shape(X_p)
        
        ones = np.array([np.ones(self.N)])
        self.X = np.concatenate((ones,X_p),axis = 0)
        self.d,self.N = np.shape(self.X)
        
        y_pred = []
        Wt = self.W_fin.reshape((1,self.d))
        for n in range(self.N):
            class_assign = 0
            if self.sigmoid(np.dot(Wt,self.X[:,n].reshape((self.d,1))))>0.5:
                    class_assign = 1
            y_pred.append(class_assign)

        return np.array(y_pred)

In [65]:
from scipy.optimize import minimize

# Training data - two randomly-generated Gaussian-distributed clouds of points in 2d space
np.random.seed(0)
# Number of points
N = 1000
# Labels for each cluster
y = np.random.randint(low=0, high=2, size = N)
# Mean of each cluster
means = np.array([[-1, 1], [-1, 1],])
# Covariance (in X and Y direction) of each cluster
covariances = np.random.random_sample((2, 2)) + 1
# Dimensions of each point
X = np.vstack([np.random.randn(N)*covariances[0, y] + means[0, y],
               np.random.randn(N)*covariances[1, y] + means[1, y]])


# plt.scatter(X[0,y==0], X[1,y==0])
# plt.scatter(X[0,y==1], X[1,y==1])


logist = LogisticReg()
W = logist.fit(X,y)
y_pred = logist.predict(X)

#check if cost function works
#print(cost(Wt_new))
#use minimize function to check which minimum weighting is required if own code works
#minimize(cost, Wt_new)

#W_min = [1.19827237, 0.74172318]

#how to determine prob of data point, if in class1 >0.5 otherwise class 2
#prob = sigmoid(np.dot(W_min,X[:,0]))
#print(prob)

plt.scatter(X[0,y_pred==0], X[1,y_pred==0])
plt.scatter(X[0,y_pred==1], X[1,y_pred==1])
    
#print(gradient(Wt_new))
#print(hessian(Wt_new))
#print(updateW(Wt_new))

2
3
[1. 1. 1.]


ValueError: cannot reshape array of size 9 into shape (1,3)